In [12]:
import copy
import random as rd
import csv
import math
import os
import pandas as pd
import numpy as np

In [ ]:
def resetParams():
    global walking_speed  # m/s
    global routeTable
    global roadLengthTable
    global arrivalTime
    global departuresProfile
    global arrivalProfile
    walking_speed = 1  # m/s
    routeTable = {}
    roadLengthTable = {}
    arrivalTime = {}
    departuresProfile = {}
    arrivalProfile = {}

def readRouteTable(metro):
    #Route table initialization
    with open(f'UptownInputs/roadCalcs_routeTable_{metro}.csv', 'r') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            profileId = row[0]
            popSplit = float(row[2])
            route = list(filter(lambda x:x!='',row[3:]))
            if profileId not in routeTable:
                routeTable[profileId] = []
            routeTable[profileId].append(
                {
                    'popSplit': popSplit,
                    'route': route
                }
            )

def readRoadLengthTable():
#Route length initialization
    with open('UptownInputs/roadCalcs_roadLengthTable.csv', 'r') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            roadProfile = row[0]
            length = float(row[1])
            roadLengthTable[roadProfile] = {
                'length': length,
                'walking_time': round(length / walking_speed) # Calculate walking time in seconds
            }

def readArrivalTime(scenario):
#population and arrival initialization
    def populationFormatter(pop):
        try:
            return 0 if pop == '' else int(pop.replace(',', ''))
        except:
            return 0
    with open(f'UptownInputs/roadCalcs_arrivalTime_{scenario}.csv', 'r') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            profileId = row[0]
            time = float(row[3])
            population = populationFormatter(row[2])
            arrivalTime[profileId] = {
                'population': population,
                'time': time
            }

def readArrivalDepartureProfile(scenario):
#arrival profile
    with open(f'UptownInputs/roadCalcs_arrivalProfile_departures_{scenario}.csv', 'r') as csvfile:
        reader = csv.reader(csvfile)
        data = list(reader)
        headers = data[0]
        arrivalData = data[1:]

        def pctFormatter(pct):
            try:
                return abs(float(pct))
            except:
                return abs(float(str(pct.split('%')[0])))/100

        for row in arrivalData:
            profileId = row[0]
            timeProfile = row[2:]
            departuresProfile[profileId] = {}
            for idx,pct in enumerate(timeProfile):
                departuresProfile[profileId][(idx*3600)] = 0 if pct == '' else pctFormatter(pct)


def readArrivalArrivalProfile(scenario):
#arrival profile
    with open(f'UptownInputs/roadCalcs_arrivalProfile_arrivals_{scenario}.csv', 'r') as csvfile:
        reader = csv.reader(csvfile)
        data = list(reader)
        headers = data[0]
        arrivalData = data[1:]

        def pctFormatter(pct):
            try:
                return abs(float(pct))
            except:
                return abs(float(str(pct.split('%')[0])))/100

        for row in arrivalData:
            profileId = row[0]
            timeProfile = row[2:]
            arrivalProfile[profileId] = {}
            for idx,pct in enumerate(timeProfile):
                arrivalProfile[profileId][(idx*3600)] = 0 if pct == '' else pctFormatter(pct)


In [14]:
def round_down(num, divisor):
    return num - (num%divisor)

In [15]:
class person:
    def __init__(self,personIndex,timeAtNode,pathList,nodeDetail,inNewSpace):
        self.idx = personIndex
        self.curNode = pathList[0]
        self.curNodeIdx = 0
        self.timeAtCurNode = timeAtNode
        self.nodeList = pathList
        self.node_detail = nodeDetail 
        self.timeAtNextNode = timeAtNode + self.calculateTimeToNextNode()
        self.agentInNewSpace = inNewSpace

    def __str__(self):
        return f"{self.idx} {self.curNode} {self.timeAtCurNode} {self.timeAtNextNode}"

    def calculateTimeToNextNode(self):
        # print(self.node_detail)
        timeToSpendAtCurNode = self.node_detail[self.curNode]['walking_time']
        return timeToSpendAtCurNode

    def updateAgentLocation(self,curTime):
        self.agentInNewSpace = False
        if curTime == self.timeAtNextNode:
            #Reached Last Step, remove details
            if self.curNode == self.nodeList[-1]:
                self.curNode = 'end'
                self.timeAtNextNode = -1
                return
    
            self.curNodeIdx+=1
            self.curNode = self.nodeList[self.curNodeIdx]
            self.timeAtCurNode = curTime
            self.timeAtNextNode = curTime + self.calculateTimeToNextNode()
            self.agentInNewSpace = True
            return

In [16]:
print(routeTable) #node paths
print(roadLengthTable)
print(arrivalTime)
print(departuresProfile)
print(arrivalProfile)

{'1': [{'popSplit': 0.5, 'route': ['F1', 'F2']}, {'popSplit': 0.05, 'route': ['C']}, {'popSplit': 0.45, 'route': ['C', 'D', 'A4', 'A3', 'A2', 'A1']}], '2': [{'popSplit': 0.5, 'route': ['F1', 'F2']}, {'popSplit': 0.05, 'route': ['C']}, {'popSplit': 0.45, 'route': ['C', 'D', 'A4', 'A3', 'A2', 'A1']}], '3': [{'popSplit': 0.5, 'route': ['F1', 'F2']}, {'popSplit': 0.05, 'route': ['F1', 'C']}, {'popSplit': 0.45, 'route': ['F1', 'C', 'D', 'A4', 'A3', 'A2', 'A1']}], '4': [{'popSplit': 0.5, 'route': ['F1', 'F2']}, {'popSplit': 0.05, 'route': ['F1', 'C']}, {'popSplit': 0.45, 'route': ['F1', 'C', 'D', 'A4', 'A3', 'A2', 'A1']}], '5': [{'popSplit': 0.5, 'route': ['F1', 'F2']}, {'popSplit': 0.05, 'route': ['F1', 'C']}, {'popSplit': 0.45, 'route': ['F1', 'C', 'D', 'A4', 'A3', 'A2', 'A1']}], '6': [{'popSplit': 0.5, 'route': ['F1', 'F2']}, {'popSplit': 0.05, 'route': ['F1', 'C']}, {'popSplit': 0.45, 'route': ['F1', 'C', 'D', 'A4', 'A3', 'A2', 'A1']}], '7': [{'popSplit': 1.0, 'route': ['A1']}], '8': [{'

In [25]:
def runScenario(metro,scenario):
    startingTime = 0*3600 #7pm in seconds
    #simulate arrival time of people coming in a lift and moving through from one to the next and calculate occupancy of each node over time
    simulationTime = 86400+startingTime #seconds
    #Escalators
    peoplePerHour = 1500
    flowRateBucket = 60
    flowRateWidthTest = 66 # people per minute per metre
    #keep track of time when people arrive at a node, where they are and when they should leave.

    arrivalRatePerSecond = peoplePerHour / 3600
    numberOfDeparturesGenerated = {}
    numberOfArrivalsGenerated = {}
    numberOfPeople = {}
    peopleList = []
    peopleEnd = 0
    nodeOccupancies = {}
    nodePeopleCount = {}
    nodeFlowRate = {}
    nodeWidthFlowRequired = {}
    for profile in routeTable:
        numberOfDeparturesGenerated[profile] = {}
        numberOfArrivalsGenerated[profile] = {}
        for routeId,routes in enumerate(routeTable[profile]):
            numberOfDeparturesGenerated[profile][routeId] = 0
            numberOfArrivalsGenerated[profile][routeId] = 0


    for curTime in range(startingTime,simulationTime,1):
        timeBucket = round_down(curTime, 3600)
        if curTime not in nodeOccupancies:
            nodeOccupancies[curTime] = {}
            nodePeopleCount[curTime] = {}
            nodeFlowRate[curTime] = {}
            nodeWidthFlowRequired[curTime] = {}
            nodeOccupancies[curTime]['end'] = 0
            nodePeopleCount[curTime]['end'] = 0
            nodeFlowRate[curTime]['end'] = 0
            nodeWidthFlowRequired[curTime]['end'] = 0


            for key in roadLengthTable:
                nodeOccupancies[curTime][key] = 0
                nodePeopleCount[curTime][key] = nodePeopleCount[curTime-1][key] if curTime > startingTime else 0
                nodeFlowRate[curTime][key] = nodePeopleCount[curTime-1][key] - nodePeopleCount[curTime-(flowRateBucket+1)][key] if curTime > startingTime+flowRateBucket+1 else 0
                nodeWidthFlowRequired[curTime][key] = nodeFlowRate[curTime-1][key]/flowRateWidthTest if curTime > startingTime else 0

        #loop through profiles ID and generate the number of people
        for profile in departuresProfile:
            if profile not in routeTable:
                continue
            for routeId, routes in enumerate(routeTable[profile]):
                profileNodePaths = routes['route']
                if len(profileNodePaths) == 0:
                    continue
                reversedProfileNode = routes['route'][::-1]
                remainingArrivalPct = 0

                #departures over 15 mins
                numberOfDepartures = round(departuresProfile[profile][timeBucket]*routes['popSplit']*arrivalTime[profile]['population'])
                hourArrivalProfile = arrivalProfile[profile][timeBucket]
                
                nodeDetails = roadLengthTable


                #40% arrive in the first 15 mins of the hour, the remainder over the next 45 mins
                if hourArrivalProfile > 0.4:
                    remainingArrivalPct = hourArrivalProfile - 0.4
                    hourArrivalProfile = 0.4


                if curTime < timeBucket+arrivalTime[profile]['time']:
                    numberOfArrivals = round(hourArrivalProfile*routes['popSplit']*arrivalTime[profile]['population'])
                    rateOfDeparturesPeople = (numberOfDepartures/arrivalTime[profile]['time'])
                    numberOfDeparturesGenerated[profile][routeId] += rateOfDeparturesPeople

                    rateOfArrivalsPeople = (numberOfArrivals/arrivalTime[profile]['time'])
                    numberOfArrivalsGenerated[profile][routeId] += rateOfArrivalsPeople

                    personDeparturesAmount = math.floor(numberOfDeparturesGenerated[profile][routeId])
                    for personIndex in range(int(personDeparturesAmount)):
                        peopleList.append(person(len(peopleList),curTime,profileNodePaths,nodeDetails,True))

                    personArrivalAmount = math.floor(numberOfArrivalsGenerated[profile][routeId])
                    for personIndex in range(int(personArrivalAmount)):
                        peopleList.append(person(len(peopleList),curTime,reversedProfileNode,nodeDetails,True))




                if remainingArrivalPct!=0 and curTime >= timeBucket+arrivalTime[profile]['time']:
                    numberOfArrivals = round(remainingArrivalPct*routes['popSplit']*arrivalTime[profile]['population'])
                    remainingRateOfArrivalsPeople = (numberOfArrivals/(3600-arrivalTime[profile]['time']))
                    numberOfArrivalsGenerated[profile][routeId] += remainingRateOfArrivalsPeople

                    personArrivalAmount = math.floor(numberOfArrivalsGenerated[profile][routeId])
                    for personIndex in range(int(personArrivalAmount)):
                        peopleList.append(person(len(peopleList),curTime,reversedProfileNode,nodeDetails,True))


                numberOfDeparturesGenerated[profile][routeId]-=math.floor(numberOfDeparturesGenerated[profile][routeId])
                numberOfArrivalsGenerated[profile][routeId]-=math.floor(numberOfArrivalsGenerated[profile][routeId])


        newPeopleList = []
        for agentId, agent in enumerate(peopleList):
            agentInNewSpace = agent.agentInNewSpace
            if agentInNewSpace:
                nodePeopleCount[curTime][agent.curNode] += 1

            agent.updateAgentLocation(curTime)
            agentLocation = agent.curNode

            if agentLocation == 'end':
                peopleEnd += 1
                continue

            nodeOccupancies[curTime][agentLocation] += 1
            newPeopleList.append(agent)
        nodeOccupancies[curTime]['end'] = peopleEnd
        peopleList = newPeopleList

# print(nodeOccupancies)

# Get headers from one of the inner dictionaries
    fieldnames = ['time'] + list(next(iter(nodeOccupancies.values())).keys())

    with open(f'UptownOutputs/{metro}/roadCalcs_OccupancyOutput_{scenario}.csv', 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for time, node_values in nodeOccupancies.items():
            row = {'time': time}
            row.update(node_values)
            writer.writerow(row)

    with open(f'UptownOutputs/{metro}/roadCalcs_PeopleCountOutput_{scenario}.csv', 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for time, node_values in nodePeopleCount.items():
            row = {'time': time}
            row.update(node_values)
            writer.writerow(row)

    with open(f'UptownOutputs/{metro}/roadCalcs_FlowRateOutput_{scenario}.csv', 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for time, node_values in nodeFlowRate.items():
            row = {'time': time}
            row.update(node_values)
            writer.writerow(row)
    with open(f'UptownOutputs/{metro}/roadCalcs_WidthFlowRequiredOutput_{scenario}.csv', 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for time, node_values in nodeWidthFlowRequired.items():
            row = {'time': time}
            row.update(node_values)
            writer.writerow(row)

In [28]:
def runMain():
    # scenarioList = ['00','01','01M','02','02B','02BM','02M','03','04','04B','04BM','05','05B','A','AA','asianG','BBCC','BC','D','DM','esport','FIFA','megaSport','megaSportM','Olympics','OlympicsT','Opening','OpeningM']
    scenarioList = ['OlympicsT']
    # metroList = ['pre','post']
    metroList = ['pre']
    for metro in metroList:
        for scenario in scenarioList:
            print(f"Running scenario {scenario} for metro {metro}")
            resetParams()
            readRouteTable(metro)
            readRoadLengthTable()
            readArrivalTime(scenario)
            readArrivalDepartureProfile(scenario)
            readArrivalArrivalProfile(scenario)
            runScenario(metro,scenario)

In [29]:
runMain()

Running scenario OlympicsT for metro pre


In [ ]:
folder_path = 'UptownOutputs'
subfolders = ['pre','post']
prefix = 'roadCalcs_WidthFlowRequiredOutput_' 


for subfolder in subfolders:
    all_dfs = []
    headers =  []
    subfolder_path = os.path.join(folder_path, subfolder)
    if os.path.isdir(subfolder_path):
        for filename in os.listdir(subfolder_path):
            if filename.startswith(prefix):
                file_path = os.path.join(subfolder_path, filename)
                headers = list(df.columns) if not len(headers) else headers
                df = pd.read_csv(file_path)
                df.drop(columns=['time','end'], axis=1, inplace=True)
                  # Optionally track which file each row came from
                max_values = df.max(numeric_only=True)
                max_values['scenario'] = filename.split('_')[-1].split('.')[0] + '_' +subfolder
                all_dfs.append(max_values)
                

# Combine all data into a single DataFrame
    combined_df = pd.DataFrame(all_dfs,index=None)
    combined_df.to_csv(f'{folder_path}/maxWidths{subfolder}.csv', index=False)

,A1,A2,A3,A4,B,C,D,E,F1,F2,G,H,I1,I2,I3,scenario
0,19.090909,12.651515,12.651515,12.651515,0.0,13.606061,12.227273,3.106061,15.833333,13.651515,0.0,0.0,0.0,0.0,0.0,Olympics_pre
1,22.075758,22.075758,22.075758,22.075758,0.0,24.530303,22.075758,3.454545,24.530303,24.530303,0.0,0.0,0.0,0.0,0.0,02BM_pre
2,8.287879,6.560606,6.560606,6.893939,0.0,7.000000,6.893939,3.454545,9.590909,7.651515,0.0,0.0,0.0,0.0,0.0,BC_pre
3,13.075758,13.075758,13.075758,14.651515,0.0,13.363636,14.651515,2.272727,15.287879,16.303030,0.0,0.0,0.0,0.0,0.0,FIFA_pre
4,18.393939,18.393939,18.393939,18.393939,0.0,20.136364,18.106061,3.454545,27.000000,20.106061,0.0,0.0,0.0,0.0,0.0,04BM_pre
5,22.500000,22.500000,22.500000,22.500000,0.0,25.000000,22.500000,0.000000,25.000000,25.000000,0.0,0.0,0.0,0.0,0.0,Opening_pre
6,16.742424,16.742424,16.742424,16.742424,0.0,18.303030,16.454545,3.454545,18.363636,18.272727,0.0,0.0,0.0,0.0,0.0,BBCC_pre
7,20.196970,20.196970,20.196970,20.196970,0.0,22.121212,19.909091,3.454545,22.121212,22.121212,0.0,0.0,0.0,0.0,0.0,02M_pre
8,39.560606,8.469697,15.090909,15.227273,0.0,9.393939,8.181818,3.454545,17.090909,16.939394,0.0,0.0,0.0,0.0,0.0,01M_pre
9,13.530303,12.257576,12.257576,12.257576,0.0,13.318182,11.969697,3.454545,13.363636,13.287879,0.0,0.0,0.0,0.0,0.0,04_pre


,A1,A2,A3,A4,B,C,D,E,F1,F2,G,H,I1,I2,I3,scenario
0,19.090909,6.303030,6.303030,6.303030,0.0,6.818182,5.439394,3.106061,21.515152,20.469697,0.0,0.0,0.0,0.0,0.0,Olympics_post
1,20.000000,9.818182,10.121212,11.287879,0.0,12.272727,9.818182,3.454545,36.787879,36.787879,0.0,0.0,0.0,0.0,0.0,02BM_post
2,8.287879,3.833333,3.833333,3.833333,0.0,3.530303,3.060606,3.454545,12.439394,11.469697,0.0,0.0,0.0,0.0,0.0,BC_post
3,9.000000,5.833333,5.833333,6.530303,0.0,6.696970,6.545455,2.272727,20.984848,24.424242,0.0,0.0,0.0,0.0,0.0,FIFA_post
4,12.575758,8.348485,8.348485,8.348485,0.0,10.090909,8.060606,3.454545,33.621212,30.166667,0.0,0.0,0.0,0.0,0.0,04BM_post
5,16.772727,10.000000,10.000000,10.000000,0.0,12.500000,10.000000,0.000000,37.500000,37.500000,0.0,0.0,0.0,0.0,0.0,Opening_post
6,9.151515,7.606061,7.606061,7.606061,0.0,9.166667,7.318182,3.454545,27.439394,27.409091,0.0,0.0,0.0,0.0,0.0,BBCC_post
7,11.636364,9.621212,9.621212,10.333333,0.0,11.060606,8.848485,3.454545,33.181818,33.181818,0.0,0.0,0.0,0.0,0.0,02M_post
8,38.106061,3.924242,6.727273,6.772727,0.0,5.136364,3.636364,3.454545,25.454545,25.378788,0.0,0.0,0.0,0.0,0.0,01M_post
9,12.575758,5.621212,5.621212,5.621212,0.0,6.681818,5.333333,3.454545,19.984848,19.939394,0.0,0.0,0.0,0.0,0.0,04_post
